In [67]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import re
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

def search_books(key='한글'):
# 환경 변수에서 값 읽기
    client_id = os.getenv("NAVER_CLIENT_ID")
    client_secret = os.getenv("NAVER_CLIENT_SECRET")

    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret,
    }

    payload = {
        'query': key,
        'display': 50,
        'sort': 'date'
    }

    url = 'https://openapi.naver.com/v1/search/book.json'

    # 요청 보내기
    res = requests.get(url, params=payload, headers=headers)
    items_data = res.json()['items']

    # 결과 저장 리스트
    items_list = list()
    item_list = []


    for i, item in enumerate(items_data):
        title = re.sub(r'<.*?>', '', item['title'])
        description = re.sub(r'<.*?\ㅜ>', '', item['description']).replace('\n', ' ')

        item_list.append(title)
        item_list.append(item['link'])
        item_list.append(item['image'])
        item_list.append(item['author'])
        item_list.append(item['pubdate'])
        item_list.append(item['discount'])
        item_list.append(item['publisher'])
        item_list.append(item['isbn'])
        item_list.append(description)

        items_list.append(item_list)
        item_list = [] 

    df = pd.DataFrame(
        items_list,
        columns=["title", "link", "image", "author", "pubdate", "discount", "publisher", "isbn", "description"]
    )

    # 저장 ->ai의 힘을 빌렸습니다..
    df.to_json('data/books.json', orient='records', force_ascii=False, indent=2)

    # 불러오기
    book_df = pd.read_json('data/books.json')
        # 출력

    return book_df


df = search_books('파이썬')
result1 = (
    df[df['discount'] >= 20000]  
    .loc[:, ['title', 'author', 'discount', 'publisher', 'pubdate']] 
    .sort_values(by='discount', ascending=False) 
    .reset_index(drop=True)  
)
publish_df = df[df['publisher'].str.contains('자유아카데미', case=False, na=False)]
 
result2= publish_df.drop(columns=['image', 'description']).reset_index(drop=True)

print(result2)

                                title  \
0               누구나 쉽게 시작하는 파이썬 프로그래밍   
1                       파이썬을 이용한 통계수학   
2     내공의 파이썬 (한 권으로 끝내는 초보자용 코딩 입문서)   
3          프로그래밍으로 배우는 양자컴퓨팅 with 파이썬   
4  파이썬을 이용한 확률과 통계 (실습 위주로 배우는 코딩 학습)   

                                                link   author   pubdate  \
0  https://search.shopping.naver.com/book/catalog...      이충규  20250320   
1  https://search.shopping.naver.com/book/catalog...      권세혁  20250315   
2  https://search.shopping.naver.com/book/catalog...      한선관  20250315   
3  https://search.shopping.naver.com/book/catalog...  임은진^권용경  20250305   
4  https://search.shopping.naver.com/book/catalog...  민만식^서준혁  20250120   

   discount publisher           isbn  
0     33320    자유아카데미  9791158086985  
1     21560    자유아카데미  9791158086992  
2     24300    자유아카데미  9791158087005  
3     29440    자유아카데미  9791158086541  
4     29440    자유아카데미  9791158086688  
